In [28]:
import chess.pgn
import pandas as pd
from tqdm import tqdm
import io
import time

In [29]:
def get_eval(game,n = None, t = None):
    # Note - I would like to make a class which contains all important metrics for my neural net
    board = game.board()
    move_number = 0
    white = []
    black = []
    centipawns = []
    engine = chess.engine.SimpleEngine.popen_uci("stockfish_15_win_x64_avx2\\stockfish_15_x64_avx2.exe")
    engine.configure({"Threads": 8}) # or even higher

    for move in game.mainline_moves():
        board.push(move)
        if move_number == 0:
            info = engine.analyse(board, chess.engine.Limit(depth = n,time=t))
            prev_centipawn = info["score"].white().score()
        if move_number > 0:
            info = engine.analyse(board, chess.engine.Limit(depth = n,time = t))
            centipawn = info["score"].white().score()
            if centipawn is None:
                centipawn = 0
            if move_number % 2 == 0:
                white.append(centipawn - prev_centipawn)
            else:
                black.append(- (centipawn - prev_centipawn))
            prev_centipawn = centipawn
            centipawns.append(centipawn)
        move_number +=1
    engine.quit()
    return white, black, centipawns

In [30]:
class Eval:
    def __init__(self, game, n, time_limit):
        self.game = game
        self.white_eval,self.black_eval,self.centipawns  = get_eval(self.game, n, time_limit)

In [31]:
def to_eval(x,n, t):
    eval = Eval(x,n,t )
    return eval

In [32]:
def pgn_to_csv(player, time_limit):
    i = 0
    file2 = open('pgns/' +player + '.pgn',encoding="utf-8",errors = "ignore")

    #TODO - change this to a more standard while loop
    game = chess.pgn.read_game(file2)
    df = pd.read_csv('blitz/' + player +'.csv')
    df = df[["White", "Black", "WhiteELO", "BlackELO", "Date","Event", "Result", "Opening", "Rounds", "Game", "Eval", "EventType"]]
    j = len(df.index)
    while isinstance(game,chess.pgn.Game):
        white = []
        black = []
        white_elo = []
        black_elo = []
        date = []
        event = []
        result = []
        wdiff = []
        bdiff = []
        rounds = []
        opening = []
        games = []
        evals = []
        i +=1
        if i <= j:
            game = chess.pgn.read_game(file2)
            continue
        start = time.time()
        print(i)
        if "White" in game.headers:
            white.append(game.headers["White"])
        else:
            white.append('*')
        if "Round" in game.headers:
            rounds.append(game.headers["Round"])
        else:
            white.append('*')
        if "Black" in game.headers:
            black.append(game.headers["Black"])
        else:
            black.append('*')
        if "WhiteElo" in game.headers:
            white_elo.append(game.headers["WhiteElo"])
        else:
            white_elo.append('*')
        if "BlackElo" in game.headers:
            black_elo.append(game.headers["BlackElo"])
        else:
            black_elo.append('*')
        if "Date" in game.headers:
            date.append(game.headers["Date"])
        else:
            date.append('*')
        if "EventType" in game.headers:
            event.append(game.headers["EventType"])
            if 'blitz' in game.headers["EventType"]:
                evals.append(to_eval(game ,20, time_limit).centipawns)
            else:
                evals.append('')
        else:
            event.append('*')
            evals.append('')
        if "ECO" in game.headers:
            opening.append(game.headers["ECO"])
        else:
            opening.append('*')
        if "Result" in game.headers:
            result.append(game.headers["Result"])
        else:
            result.append('*')
        games.append(game.mainline_moves())
        #evals.append(to_eval(game ,20).centipawns)
        end = time.time()
        print(f"time elapsed: {end - start}")
        game = chess.pgn.read_game(file2)

        data = {"White": white, "Black": black, "WhiteELO": white_elo, "BlackELO": black_elo, "Date": date,"Event": event, "Result": result, "Opening": opening, "Rounds": rounds, "Game": games, "Eval": evals, "EventType": event}

        df1 = pd.DataFrame(data)

        df = pd.concat([df,df1])


        df = df.reset_index(drop=True)

        df.to_csv('blitz/' + player +'.csv')

In [33]:
players = ['andreikin, dmitry', 'anand, viswanathan', 'wang, hao', 'grischuk, alexander', 'karjakin, sergey','duda, jan-krzysztof', 'radjabov, teimour', 'dominguez perez, leinier','nakamura, hikaru', 'vachier-lagrave, maxime','aronian, levon','mamedyarov, shakhriyar', 'so, wesley','ding, liren','carlsen, magnus', 'rapport, richard', 'nepomniachtchi, ian', 'giri, anish', 'firouzja, alireza', 'caruana, fabiano','zelcic, robert','khotenashvili, bela', 'bischoff, klaus', 'hoffmann, asa','kaufman, lawrence','bellaiche, elise']
players.reverse()
for player in players:
    print(player)
    pgn_to_csv(player, 600)

bellaiche, elise
kaufman, lawrence
hoffmann, asa
1698
time elapsed: 143.66604948043823
1699
time elapsed: 0.0
1700
time elapsed: 0.0
1701
time elapsed: 0.0
1702
time elapsed: 0.0
1703
time elapsed: 0.0
1704
time elapsed: 0.0
1705
time elapsed: 0.0
1706
time elapsed: 167.95303344726562
1707
time elapsed: 0.0
1708
time elapsed: 0.0
1709
time elapsed: 0.0
1710
time elapsed: 0.0
1711
time elapsed: 0.0
1712
time elapsed: 194.04638385772705
1713
time elapsed: 263.07665157318115
1714
time elapsed: 0.0
1715
time elapsed: 815.1566348075867
1716
time elapsed: 168.00074744224548
1717
time elapsed: 0.0
1718
time elapsed: 0.0
1719
time elapsed: 0.010019779205322266
1720
time elapsed: 0.0
1721
time elapsed: 0.0
1722
time elapsed: 0.0
1723
time elapsed: 0.0
1724
time elapsed: 0.0
1725
time elapsed: 0.0
1726
time elapsed: 0.0
1727
time elapsed: 0.0
1728
time elapsed: 0.0
1729
time elapsed: 0.0
1730
time elapsed: 0.0
1731
time elapsed: 0.0
1732
time elapsed: 157.270263671875
1733
time elapsed: 366.8118


KeyboardInterrupt



In [ ]:
# data = {"White": [], "Black": [], "WhiteELO": [], "BlackELO": [], "Date": [],"Event": [], "Result": [], "Opening": [], "Rounds": [], "Game": [], "Eval": [], "EventType": []}
#
# df = pd.DataFrame(data)
# for player in players:
#     df.to_csv('blitz/' + player + '.csv')